In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import resample_poly
import random
#from scipy.fft import fft, fftfreq
#from sklearn.preprocessing import MinMaxScaler

%matplotlib tk


In [23]:
# Functions -------------------------------------------------------------

def plot_line(series):
    plt.figure(figsize=(10, 5))
    plt.plot(series.index, series.values)
    plt.show()


def polyphase_resample(df, target_fs=16):
    # Step 1: Extract time and signal values
    time_s = df['time'].values           # Time in milliseconds
    signal = df['value'].values              # Signal samples

    # Step 2: Convert time to seconds for consistency with sampling rates
    #time_s =  / 1000.0

    # Step 3: Estimate average sample rate from time differences
    dt = np.diff(time_s)                     # Time intervals between samples
    avg_dt = np.mean(dt)                     # Average time step
    avg_fs = 1.0 / avg_dt                    # Average sample rate (in Hz)
    #print(dt)
    print(f"Estimated average sampling rate: {avg_fs:.2f} Hz, with average dt: {avg_dt:.6f} seconds")

    # Step 4: Decide target sample rate (e.g., exact 25 Hz)
    gcd = np.gcd(int(round(avg_fs * 1000)), int(target_fs * 1000))  # Prevent fraction explosion
    up = int(round(target_fs * 1000 / gcd))                         # Upsample factor
    down = int(round(avg_fs * 1000 / gcd))                          # Downsample factor
    print(f"Resampling with up={up}, down={down}")

    # Step 5: Resample using polyphase filtering
    resampled_signal = resample_poly(signal, up, down)

    # Step 6: Generate new uniform time axis for resampled signal
    #duration = time_s[-1] - time_s[0]                            # Total signal duration in seconds
    num_samples = len(resampled_signal)
    uniform_time = np.linspace(time_s[0], time_s[-1], num=num_samples)
    resampled_df = pd.DataFrame({'time': uniform_time, 'value': resampled_signal})
    resampled_df.reset_index(drop=True, inplace=True)
    #print(f"Resampled signal length: {len(resampled_df)} samples")
    return resampled_df


def addsym(x, y, sym, num_samples_sym, orig_x_df, orig_y_df):
    if sym:
        start_idx = np.random.randint(0, len(orig_x_df) - (num_samples_sym+1))
        x.extend(orig_x_df['value'].iloc[start_idx:start_idx + num_samples_sym].values)
        y.extend(orig_y_df['value'].iloc[start_idx:start_idx + num_samples_sym].values)
    else:
        x.extend([0] * num_samples_sym)
        y.extend([0] * num_samples_sym)
    return x, y


def generate_synth_data(adc1: pd.DataFrame,
                        adc2: pd.DataFrame,
                        n_bits: int = 100,
                        col: str = "value",
                        seed: int | None = None) -> pd.DataFrame:
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    on_len  = 16        # samples per symbol
    symbols = []        # rows will accumulate here

    curr_state = 0  # start state doesn’t really matter—will flip on first bit

    # precompute valid starting indices for contiguous 16‑sample blocks
    max_start1 = len(adc1) - on_len
    max_start2 = len(adc2) - on_len
    if max_start1 < 0 or max_start2 < 0:
        raise ValueError("adc1/adc2 must each have ≥16 rows")

    sample_counter = 0
    for bit_idx in range(n_bits):
        bit_val = random.randint(0, 1)          # 0 → two transitions, 1 → one transition
        # Determine how many state changes we need *within* this bit

        #n_transitions = 2 if bit_val == 0 else 1

        for sym_idx in range(2):                # exactly two symbols per bit
            # flip state at start of symbol if we still owe transitions this bit
            # if n_transitions > 0:
            #     curr_state = 1 if curr_state == 0 else 0
            #     n_transitions -= 1

            if (sym_idx == 0 and bit_val == 0) or sym_idx == 1:
                curr_state = not curr_state  # flip state for zero bits

            # build 16‑sample slice
            if curr_state == 1:
                i1 = random.randint(0, min(max_start1, max_start2))
                xs = adc1[col].iloc[i1 : i1 + on_len].to_numpy()
                ys = adc2[col].iloc[i1 : i1 + on_len].to_numpy()
            else:  # OFF
                xs = np.zeros(on_len)
                ys = np.zeros(on_len)

            for k in range(on_len):
                symbols.append({
                    "sample_num": sample_counter,
                    "x":          int(xs[k]),
                    "y":          int(ys[k]),
                    "bit_num":    bit_idx,
                    "bit":        bit_val,
                    "sym_num":    sym_idx,
                    "sym":        1 if curr_state else 0
                })
                sample_counter += 1

    synth_data = pd.DataFrame(symbols,
                              columns=["sample_num","x","y",
                                       "bit_num","bit","sym_num","sym"])
    return synth_data

In [6]:
# Main -------------------------------------------------------------------

# Load the file
df = pd.read_csv("../data_to_keep/may23-proc.txt")

# Split the dataframe into two based on the 'adc' column
adc1 = df[df['adc'] == 1].copy()
adc2 = df[df['adc'] == 2].copy()

#change samples to seconds
adc1['time'] = adc1['time'] / 1000.0  # Convert milliseconds to seconds
adc2['time'] = adc2['time'] / 1000.0  # Convert milliseconds to seconds

# Reset the 'time' column to start from zero for both adc1 and adc2, and sort by 'time'
adc1.loc[:, "time"] = adc1["time"] - adc1["time"].iloc[0]
adc2.loc[:, "time"] = adc2["time"] - adc2["time"].iloc[0]
adc1 = adc1.sort_values(by='time').reset_index(drop=True)
adc2 = adc2.sort_values(by='time').reset_index(drop=True)

# Check data
print("Data for ADC1:")
print(adc1.head())
print("Data for ADC2:")
print(adc2.head())
print("ADC1 Maximum Value:", adc1['value'].max())
print("ADC2 Maximum Value:", adc2['value'].max())

Data for ADC1:
    time  adc     value
0  0.000    1  3644.625
1  0.039    1  3225.000
2  0.078    1  3050.375
3  0.117    1  3216.750
4  0.350    1  3581.375
Data for ADC2:
    time  adc     value
0  0.000    2  1107.000
1  0.039    2  1100.375
2  0.078    2  1098.750
3  0.117    2  1100.875
4  0.156    2  1098.750
ADC1 Maximum Value: 4095.875
ADC2 Maximum Value: 1109.25


In [9]:
resampled_adc1 = polyphase_resample(adc1, target_fs=16)
resampled_adc2 = polyphase_resample(adc2, target_fs=16)



# Optional: visualize original and resampled
plt.figure(figsize=(12, 6))
plt.plot(resampled_adc1["time"], resampled_adc1["value"], label='ADC1', alpha=0.8)
plt.plot(adc1["time"], adc1["value"], label='Original ADC1', alpha=0.8, linestyle='--')

plt.plot(resampled_adc2["time"], resampled_adc2["value"], label='ADC2', alpha=0.8)
plt.plot(adc2["time"], adc2["value"], label='Original ADC2', alpha=0.8, linestyle='--')
plt.xlabel('Time (s)')
plt.ylabel('Signal')
plt.title('Resampled Data vs Original')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

Estimated average sampling rate: 12.65 Hz, with average dt: 0.079061 seconds
Resampling with up=2000, down=1581
Estimated average sampling rate: 6.35 Hz, with average dt: 0.157586 seconds
Resampling with up=8000, down=3173


In [24]:
synth_data = generate_synth_data(adc1, adc2, n_bits=500, seed=42)

synth_data.to_csv("synth_data_gen.csv", index=False)



In [ ]:

# # Original signal and its (irregular) time base
# original_time = df['time'].values / 1000.0
# original_signal = df['value'].values

# # Resampled signal and uniform time base
# resampled_signal = resampled_signal
# resampled_time = uniform_time  # Already delay-compensated

# # Use interpolated FFT for irregularly sampled data (Lomb-Scargle or zero-padded FFT)

# # Estimate power spectrum of original signal (irregular sampling)
# frequencies = np.linspace(0.01, 12.5, 500) * 2 * np.pi  # Angular frequencies (radians/sec)
# psd_orig = lombscargle(original_time, original_signal, frequencies)

# # FFT of resampled signal (uniform sampling)
# # N = len(resampled_signal)
# # fft_vals = np.abs(fft(resampled_signal))[:N // 2]
# # fft_freqs = fftfreq(N, d=1/target_fs)[:N // 2]
# fft_freqs = lombscargle(resampled_time, resampled_signal, frequencies)
# print(fft_freqs)

# # Plot
# plt.figure(figsize=(12, 5))
# plt.plot(frequencies / (2*np.pi), psd_orig, label='Original (Lomb-Scargle)', alpha=0.7)
# #plt.plot(fft_freqs, fft_vals, label='Resampled (FFT)', linewidth=2)
# plt.plot(frequencies / (2*np.pi), fft_freqs, label='Resampled (FFT)', linewidth=2, alpha=0.7)
# plt.xlabel("Frequency (Hz)")
# plt.ylabel("Amplitude")
# plt.title("Spectral Comparison")
# plt.grid(True)
# plt.legend()
# plt.tight_layout()
# plt.show()


In [ ]:
# fft_result = np.fft.fft(resampled_signal)
# frequencies = np.fft.fftfreq(len(resampled_signal), d=uniform_time[1] - uniform_time[0])  # d = time step

# positive_freqs = frequencies[:len(frequencies)//2]
# magnitude = np.abs(fft_result)[:len(frequencies)//2] * 2 / len(signal)

# plt.plot(positive_freqs, magnitude)
# plt.title("Frequency Spectrum")
# plt.xlabel("Frequency (Hz)")
# plt.ylabel("Magnitude")
# plt.grid(True)
# plt.show()